## "Бич-трэвел". Описание проекта.

**Краткое описание.** Как известно, перемещения внутри Европы зачастую очень дешёвые (легко найти билеты лоукостеров вроде RyanAir за 6-10 евро или доехать на автобусе). Однако поисковики авиабилетов не предоставляют опции (во всяком случае, у меня не получилось найти) поиска маршрутов с разными промежуточными городами (только если заранее знать эти города, что не совсем наш случай). 

Моей задачей было сделать программу которая ищет комбинированные перелёты, пользуясь спец.предложениями величайшей российской авиакомпании (Победа) и предлагает самые дешёвые из них. Программа подойдёт тем, кому принципиальна цена путешествия или просто тем, кто любит путешествовать по Европе (но впрочем, никто не мешает запустить программу для какого-нибудь российского города) без какой-то конкретной цели.

**Программа состоит из трёх частей.**
1. Первая часть ищет комбинированные перелёты на сайте Победы. Замечу сразу, что эта часть является нестабильной, потому что спецпредложения постоянно меняют (по неизвестному мне принципу) и то, что работает сегодня, может спокойно не работать завтра. Поэтому не стоит сильно расстраиваться, если вдруг программа выдала, что комбинаций перелётов не найдено. Это скорее всего значит не то, что программа не работает, а то, что сегодня не ваш (и не мой) день. *На момент начала проекта Победа искала до конца мая. На момент сдачи проекта (15.04), Победа адекватно ищет только до конца апреля. Возможно, на момент проверки станет получше, но ничего гарантировать, как вы понимаете, не могу.*
2. Вторая часть ищет самую дешёвую комбинацию из двух прямых перелётов в город назначения с помощью сайта Aviasales и выводит самый дешёвый вариант на выбранные вами даты. Тут так же стоит заметить, что Авиасейлс может выдавать разные цены для разных пользователей (что как-то странно, но проверено мной при отладке программы), поэтому не могу дать гарантии, что если вы потом захотите купить эти билеты, то их стоимость будет в точности совпадать с озвученной программой (но отличаться, впрочем, будет несильно). *Если вы ввели город, в который в какие-то из дней нет перелётов, то программа может работать некорректно, так как данные о цене не берутся численно (чисел в графике на сайте нигде нет), а высчитываются из высоты столбцов, поэтому в данных появятся несуществующие низкие цены.*
3. Третья (бонусная) часть выдаёт вам погоду (потому что это прикольно и потому что мне нужно было побольше использовать API) в месте вашего путешествия (было решено, что репрезентативнее будет выдавать не текущую погоду, а погоду предыдущего года в даты предлагаемого путешествия), а также содержит анализ и размышления о корреляции стоимости билетов и времени до вылета.

**Размышления о корреляции.**
Существует довольно известное утверждение, что при формировании цен на билеты авиакомпании осуществляют ценовую дискриминацию, повышая стоимость билетов по мере приближения к дате вылета. Пользуясь случаем, было решено проверить, насколько сильно коррелирует стоимость билетов с количеством дней, оставшихся до вылета (по предположению, корреляция должна была быть близка к -1). Однако, в большинстве случаев, если брать данные с апреля до октября-ноября, то корреляция близка к нулю. Но если брать 30-, а ещё лучше 10-дневный период от текущей даты, то предполагавшаяся обратная зависимость действительно присутствует.

**За что можно поставить баллы?**
Для удобства оценивания, все функции в файле с программой расположены в алфавитном порядке.

### Pandas
Датафреймы используются в первой части для реализации алгоритма поиска самых дешёвых комбинаций.

In [1]:
def flights_city(city, all_fl):
    res = pd.concat([all_fl[all_fl['Откуда'] == city], all_fl[all_fl['Куда'] == city]], ignore_index = True)
    return res

def get_all_flights(page):
    all_flights = pd.DataFrame(columns = ['Откуда', 'Куда', 'Цена', 'Дата'])
    for f in page.findAll('li', class_ = 'airtickets-item clearfix'):
        flight = str(f.find('div', class_ = 'airtickets-cities'))
        flight = flight.strip('<div class="airtickets-cities"></')
        flight = flight.split('<br/>')
        price = int(f.find('div', class_ = 'airtickets-cost').text.strip('руб').replace(' ', ''))
        date = f.find('div', class_ = 'airtickets-date').text
        date = date_converter(date)
        if len(flight) == 2:
            fl_info = pd.DataFrame([[flight[0], flight[1], price, date]], columns = ['Откуда', 'Куда', 'Цена', 'Дата'])
            all_flights = pd.concat([all_flights, fl_info], ignore_index = True)
    return all_flights

# из main()
pos_flights = pd.DataFrame(columns = ['Откуда', 'Куда', 'Цена', 'Дата'])
for air in airports:
    pos_flights = pd.concat([pos_flights, flights_city(air, all_fl)], ignore_index = True)
pos_flights = flights_city('Москва (Внуково)', pos_flights)
fl_first = pos_flights[pos_flights['Откуда'] == 'Москва (Внуково)']
fl_second = pos_flights[pos_flights['Куда'] == 'Москва (Внуково)']

comb = fl_first.merge(fl_second, left_on = 'Откуда', right_on = 'Куда')
comb = comb[comb['Дата_y'].apply(toord) - comb['Дата_x'].apply(toord) >= min_spot]
comb = comb[comb['Дата_y'].apply(toord) - comb['Дата_x'].apply(toord) <= max_spot]
comb['Итоговая цена'] = comb['Цена_x'] + comb['Цена_y']
comb = comb.sort_values(by = 'Итоговая цена')

NameError: name 'pd' is not defined

### Веб-скреппинг
В большом количестве использовался BeautifulSoup. Используется selenium для получения информации с сайта Победы (который весь меняется динамически без перехода на другую страницу) и для обхода защиты баз 2004 года, но более содержательное использование для получения графика всех цен с Aviasales. Ну и главное детище (которое, к сожалению, так и не удалось доработать до конца, но которое всё равно выглядит довольно внушительно): кликание по датам на сайте Победы (функция pobeda_class).

In [ ]:
def check_russian(c):
    r = requests.get('http://www.avia-shop.ru/codes.phtml?country=RU')
    page = BeautifulSoup(r.text, features = 'html')
    body = page.findAll('table')[9]
    for i in body.findAll('tr'):
        if i.find('td').text.split()[0] == c:
            return i.find('td').text.split()[1].strip('()')
    return '404'

def generate_coordinates(page):
    cities = page.findAll('div', class_ = 'form-dropoutList')[1]
    all_cities = []
    for c in cities.findAll('div', class_ = 'form-dropoutList__item'):
        city = c.text.strip()
        req = requests.get('https://geocode-maps.yandex.ru/1.x/?geocode=' + city)
        page_city = BeautifulSoup(req.text, features = 'xml')
        coords = page_city.Point.pos.text.split()
        coords[0] = float(coords[0])
        coords[1] = float(coords[1])
        all_cities.append([city, coords])
    return all_cities

def get_airport_code(c):
    browser = webdriver.Chrome('C:\Program Files\chromedriver.exe')
    browser.get('http://www.aviation.com.ua/sirena/goroda.htm')
    bs = BeautifulSoup(browser.page_source)
    browser.close()
    all_air = bs.findAll('pre')[2].text.split('\n')[0:1022]
    res = []
    for city in all_air:
        city = city.split('\t')
        if city[1].count(c) > 0:
            res.append(city)
    if len(res) == 0:
        check = check_russian(c)
        if check == '404':
            return '404'
        else:
            return check
    else:
        res.sort(key = second_elem)
        return res[0][0]

def get_all_flights(page):
    all_flights = pd.DataFrame(columns = ['Откуда', 'Куда', 'Цена', 'Дата'])
    for f in page.findAll('li', class_ = 'airtickets-item clearfix'):
        flight = str(f.find('div', class_ = 'airtickets-cities'))
        flight = flight.strip('<div class="airtickets-cities"></')
        flight = flight.split('<br/>')
        price = int(f.find('div', class_ = 'airtickets-cost').text.strip('руб').replace(' ', ''))
        date = f.find('div', class_ = 'airtickets-date').text
        date = date_converter(date)
        if len(flight) == 2:
            fl_info = pd.DataFrame([[flight[0], flight[1], price, date]], columns = ['Откуда', 'Куда', 'Цена', 'Дата'])
            all_flights = pd.concat([all_flights, fl_info], ignore_index = True)
    return all_flights

def get_graph(code1, code2, d1):
    browser = webdriver.Chrome('C:\Program Files\chromedriver.exe')
    browser.get('https://www.aviasales.ru/search/' + code1 + d1 + code2 + '1')
    time.sleep(20)
    exp = browser.find_element_by_class_name('prediction-header__expand-button')
    exp.click()
    time.sleep(2)
    switch = browser.find_elements_by_class_name('calendar-type-switcher__item')[1]
    switch.click()
    time.sleep(2)
    bs = BeautifulSoup(browser.page_source)
    browser.close()
    pr = bs.find('div', class_ = 'graph__y-item', style = 'top: 0%;')
    max_price = int(pr.find('span').text.replace('\u2009', ''))
    graph = bs.find('svg', class_ = 'graph__axis-x')
    prices = []
    for rec in graph.findAll('rect')[1:750:3]:
        prices.append(int((100 - float(rec['height'].strip('%'))) * max_price / 100))
    return prices

def pobeda_class():
    browser = webdriver.Chrome('C:\Program Files\chromedriver.exe')
    browser.get('https://www.pobeda.aero/information/book/search_cheap_tickets')
    time.sleep(2)
    
    date_from = browser.find_element_by_name('date_departure_from')
    date_to = browser.find_element_by_name('date_departure_to')
    date_from.click()
    time.sleep(2)
    
    bs = BeautifulSoup(browser.page_source)
    ident = bs.find('div', class_ = 'header-form__calendar hasDatepicker')['id']
    ident = '"dp' + str(int(ident.strip('dp')) + 10) + '"'
    
    today = datetime.now().date()
    num_clicks = d1.month - today.month
    
    for i in range(num_clicks):
        browser.find_element_by_xpath('//*[@id='+ ident +']/div[1]/div[2]/div/a').click()
    
    browser.find_element_by_xpath('//*[@id='+ ident +']/div[1]/div[1]/table/tbody//*[text()="' + str(d1.day) + '"]').click()
    time.sleep(5)
    elem = browser.find_element_by_xpath('//*[@id='+ ident +']')
    ref = elem.get_attribute('outerHTML')
    mon = ref.split('data-end=')[1].split()[0]
    mon = int(mon.split('.')[1])
    if mon - int(d2.month) < 0:
        for i in range(int(d2.month) - mon + 1):
            browser.find_element_by_xpath('//*[@id='+ ident +']/div[1]/div[2]/div/a').click()
    elif mon - int(d2.month) > 0:
        for i in range(mon - int(d2.month) + 1):
            browser.find_element_by_xpath('//*[@id='+ ident +']/div[1]/div[1]/div/a').click()
    browser.find_element_by_xpath('//*[@id='+ ident +']/div[1]/div[1]/table/tbody//*[text()="' + str(d2.day) + '"]').click()

### API
API использовался для получения координат городов (как в домашке, через геокодер яндекса) и для делания всяких прикольных штук с погодой через API MetaWeather (включая вывод соответствующей картинки с погодой).

In [ ]:
def generate_coordinates(page):
    cities = page.findAll('div', class_ = 'form-dropoutList')[1]
    all_cities = []
    for c in cities.findAll('div', class_ = 'form-dropoutList__item'):
        city = c.text.strip()
        req = requests.get('https://geocode-maps.yandex.ru/1.x/?geocode=' + city)
        page_city = BeautifulSoup(req.text, features = 'xml')
        coords = page_city.Point.pos.text.split()
        coords[0] = float(coords[0])
        coords[1] = float(coords[1])
        all_cities.append([city, coords])
    return all_cities

def get_coord(city):
    req = requests.get('https://geocode-maps.yandex.ru/1.x/?geocode=' + city)
    page_city = BeautifulSoup(req.text, features = 'xml')
    coords = page_city.Point.pos.text.split()
    coords[0] = float(coords[0])
    coords[1] = float(coords[1])
    return coords

def get_woeid(city):
    coords = get_coord(city)
    params = {'action':'query', 'format': 'json'}
    r = requests.get('https://www.metaweather.com/api/location/search/?lattlong=' + str(coords[1])+','+str(coords[0]), params)
    res = re.findall(r'\{[^\}]+', r.text)[0] + '}'
    dic = json.loads(res)
    city_eng = dic['title']
    woeid = dic['woeid']
    return woeid

def get_weather(woeid, date):
    r1 = requests.get('https://www.metaweather.com/api/location/' + str(woeid) + '/' + str(date.year) + '/' + str(date.month) + '/' + str(date.day))
    page = BeautifulSoup(r1.text, features = 'html')
    res1 = re.findall(r'\{[^\}]+', page.text)[0] + '}'
    dic1 = json.loads(res1)
    return {'weather': dic1['weather_state_name'], 
           'pic' : dic1['weather_state_abbr'],
           'min_temp' : dic1['min_temp'],
           'max_temp' : dic1['max_temp']}

def draw_pic(ab):
    display(Image(url = 'https://www.metaweather.com/static/img/weather/png/' + ab + '.png', width = 50, height = 50))

### Визуализация
Красивые графики, построенные с помощью iplot в сочетании с cufflinks.

In [ ]:
def plot_prices(p1, p2, min_date, p1_full):
    init_notebook_mode()
    cufflinks.go_offline()
    
    t = [0] * len(p1)
    for i in range(len(p1)):
        t[i] = datetime.fromordinal(min_date.toordinal() + i).date()
    d1 = pd.Series(p1, index = t)
    d2 = pd.Series(p2, index = t)
    data = pd.DataFrame([d1, d2], index = ['Из Москвы', 'В Москву']).transpose()
    
    data[['Из Москвы', 'В Москву']].iplot(kind = 'bar', barmode = 'overlay', xTitle='Дата',
                                          yTitle='Стоимость билета (в рублях)', title = 'График стоимости билетов')

    bins = np.arange(0, len(p1_full))
    p1 = np.array(p1)
    mean_1 = int(np.mean(p1))
    median_1 = np.median(p1)
    corr_1 = scst.pearsonr(p1_full, bins)[0]
    corr_2 = scst.pearsonr(p1_full[0:30], bins[0:30])[0]
    corr_3 = scst.pearsonr(p1_full[0:10], bins[0:10])[0]
    print('Немного статистики по стоимости билетов из Москвы в город назначения:')
    print('Средняя стоимость билетов: ' + str(mean_1) + ' рублей, медианная стоимость: ' + str(median_1) + ' рублей')
    print('\nЛогично предположить, что цена билета должна отрицательно зависеть от количества дней до отправления.')
    print('Давайте проверим эту гипотезу, посчитав коэффициент корреляции')
    print('Коэффициент корреляции (для всех цен): ' + str(corr_1))
    print('Коэффициент корреляции (для вылета в течение 30 ближайших дней): ' + str(corr_2))
    print('Коэффициент корреляции (для вылета в течение 10 ближайших дней): ' + str(corr_3))
    print('Размышления о корреляции можно найти в описании проекта.')
    
    data['Из Москвы'].iplot(kind = 'hist', xTitle='Стоимость билета (в рублях)',
                                          yTitle='Кол-во дней с самым дешёвым билетом по данной цене', 
                                          title = 'Распределение стоимости билетов из Москвы в город назначения')

### Математика
Использовались массивы numpy для оптимизации алгоритма по вычислению расстояния между двумя точками на сфере и scipy.stats для проведения небольшого исследования о корреляции.

In [ ]:
def distance(coord1, coord2):
    rad = 6371 * 1000
    
    coords = np.array([coord1[1], coord2[1], coord1[0], coord2[0]])
    coords = coords * math.pi/180
    
    delta_long = coords[3] - coords[2]
    
    lat = coords.copy()
    lat[0:2] = np.cos(coords[0:2]).copy()
    lat[2:4] = np.sin(coords[0:2]).copy()
    
    nom = math.pow(lat[1] * math.sin(delta_long), 2) + math.pow(lat[0] * lat[3] - lat[2] * lat[1] * math.cos(delta_long), 2)
    nom = math.sqrt(nom)
    den = lat[2] * lat[3] + lat[0] * lat[1] * math.cos(delta_long)
    angle_dif = math.atan2(nom, den)
    return angle_dif * rad

def plot_prices(p1, p2, min_date, p1_full):
    init_notebook_mode()
    cufflinks.go_offline()
    
    t = [0] * len(p1)
    for i in range(len(p1)):
        t[i] = datetime.fromordinal(min_date.toordinal() + i).date()
    d1 = pd.Series(p1, index = t)
    d2 = pd.Series(p2, index = t)
    data = pd.DataFrame([d1, d2], index = ['Из Москвы', 'В Москву']).transpose()
    
    data[['Из Москвы', 'В Москву']].iplot(kind = 'bar', barmode = 'overlay', xTitle='Дата',
                                          yTitle='Стоимость билета (в рублях)', title = 'График стоимости билетов')

    bins = np.arange(0, len(p1_full))
    p1 = np.array(p1)
    mean_1 = int(np.mean(p1))
    median_1 = np.median(p1)
    corr_1 = scst.pearsonr(p1_full, bins)[0]
    corr_2 = scst.pearsonr(p1_full[0:30], bins[0:30])[0]
    corr_3 = scst.pearsonr(p1_full[0:10], bins[0:10])[0]
    print('Немного статистики по стоимости билетов из Москвы в город назначения:')
    print('Средняя стоимость билетов: ' + str(mean_1) + ' рублей, медианная стоимость: ' + str(median_1) + ' рублей')
    print('\nЛогично предположить, что цена билета должна отрицательно зависеть от количества дней до отправления.')
    print('Давайте проверим эту гипотезу, посчитав коэффициент корреляции')
    print('Коэффициент корреляции (для всех цен): ' + str(corr_1))
    print('Коэффициент корреляции (для вылета в течение 30 ближайших дней): ' + str(corr_2))
    print('Коэффициент корреляции (для вылета в течение 10 ближайших дней): ' + str(corr_3))
    print('Размышления о корреляции можно найти в описании проекта.')
    
    data['Из Москвы'].iplot(kind = 'hist', xTitle='Стоимость билета (в рублях)',
                                          yTitle='Кол-во дней с самым дешёвым билетом по данной цене', 
                                          title = 'Распределение стоимости билетов из Москвы в город назначения')

### Другие технологии
Использовались модули Ipython.display (вывод картинок), ipywidgets (виджеты для удобного ввода дат), re (регулярные выражения для более удобной работы с API погоды) и модуль datetime (много работы с датами).

In [ ]:
def draw_pic(ab):
    display(Image(url = 'https://www.metaweather.com/static/img/weather/png/' + ab + '.png', width = 50, height = 50))
    
print("Самая ранняя возможная дата отправления?")
widget_start = widgets.DatePicker(description='Отправление', disabled=False)
display(widget_start)

print("Самая поздняя возможная дата прибытия?")
widget_finish = widgets.DatePicker(description='Прибытие', disabled=False)
display(widget_finish)

def get_weather(woeid, date):
    r1 = requests.get('https://www.metaweather.com/api/location/' + str(woeid) + '/' + str(date.year) + '/' + str(date.month) + '/' + str(date.day))
    page = BeautifulSoup(r1.text, features = 'html')
    res1 = re.findall(r'\{[^\}]+', page.text)[0] + '}'
    dic1 = json.loads(res1)
    return {'weather': dic1['weather_state_name'], 
           'pic' : dic1['weather_state_abbr'],
           'min_temp' : dic1['min_temp'],
           'max_temp' : dic1['max_temp']}

def get_woeid(city):
    coords = get_coord(city)
    params = {'action':'query', 'format': 'json'}
    r = requests.get('https://www.metaweather.com/api/location/search/?lattlong=' + str(coords[1])+','+str(coords[0]), params)
    res = re.findall(r'\{[^\}]+', r.text)[0] + '}'
    dic = json.loads(res)
    city_eng = dic['title']
    woeid = dic['woeid']
    return woeid

def get_best(p1, p2, min_date, max_date, min_spot, max_spot):
    delta1 = min_date.toordinal() - datetime.now().date().toordinal()
    delta2 = max_date.toordinal() - datetime.now().date().toordinal()
    len_p1 = len(p1)
    len_p2 = len(p2)
    p_min = 1000000
    for i in range(delta1, min(len_p1, delta2) - min_spot):
        for j in range(min(i + min_spot, len_p2, delta2), min(i + max_spot + 1, len_p2, delta2 + 1)):
            if (p1[i] + p2[j] < p_min):
                p_min = p1[i] + p2[j]
                best = (i, j)
    return (p_min, best)

def say_res(res):
    today = datetime.now().date()
    date_start = datetime.date(datetime.fromordinal(today.toordinal() + res[1][0]))
    date1 = str(date_start)
    date2 = str(datetime.date(datetime.fromordinal(today.toordinal() + res[1][1])))
    print('Самый дешёвый вариант прямого путешествия в выбранный вами город:')
    print('Цена перелёта: ' + str(res[0]) + ' рублей, Даты: c ' + date1 + ' по ' + date2)
    return date_start